In [86]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import keras.backend as K
import skimage.io as io
import skimage.transform as tr
import matplotlib.pyplot as plt
from glob import glob
from keras.optimizers import *

from loss_metrics import *
from helper import * 

%matplotlib inline

In [444]:
# load the model
model = load_model('/home/dan/Desktop/Datenset_Group4_2 (Augmented)/results/training_01_best.h5', custom_objects={'iou_loss': iou_loss, 'f1': f1, 'iou':iou, 'precision' :precision, 'recall' : recall, 'error' : error})

In [445]:
InputDir = '/home/dan/Desktop/views/images/data/*.png'

InputMask = '/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data' + '/'
OutputDir = '/home/dan/Desktop/views/results/4_2'

# read the test-input-data
x = None
x = read_imgs(InputDir, (224,224,1))

# read the test-groundtruth-data
y = None
#y = read_imgs('/home/dan/Desktop/Datenset_Block3/test/masks/data/*.png', (224,224,1))

/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [446]:
# do the prediction
#test_image = x[2]
#test_image = np.expand_dims(test_image, axis=0)
pred = model.predict(x, verbose=1)

19/19 [==============================] - 4s 217ms/step


In [447]:
# save the images  
import scipy.misc as sc

def iterate(path, size):
    paths = glob(path)
    i = 0
    for p in paths:
        
        print(i)       
        tmp = p.replace("/home/dan/Desktop/views/images/data/", InputMask)
        print(tmp)
        
        mask = io.imread(tmp, (224,224,1))
        
        result = postProcessImage(pred[i][:,:,0])
        gt = postProcessImage(mask)
        diff = compareMasks(result, gt)
        
        imagePath = OutputDir + '/img/' + str(i) + '.png'
        maskPath =  OutputDir + '/pred/' + str(i) + '_pred.png'
        gtPath   =  OutputDir + '/gt/' + str(i) + '_gt.png'
        diffPath =  OutputDir + '/diff/' + str(i) + '_diff.png' 
        
        sc.imsave(imagePath, x[i, ... ,0])
        sc.imsave(gtPath, gt)
        sc.imsave(maskPath, result)
        sc.imsave(diffPath, diff)
        
        i += 1
           

    
       
                             
iterate(InputDir, (224,224,1))


    

0
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot78.png


/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/dan/.conda/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


1
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot101.png
2
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot81.png
3
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot972.png
4
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot923.png
5
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot137.png
6
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot702.png
7
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot917.png
8
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot1221.png
9
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot1180.png
10
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot967.png
11
/home/dan/Desktop/FINAL_DATASETS/Datenset_Group4_2/train/masks/data/screenshot673.png
12
/home/dan/Desktop/FINAL_DA

In [439]:
from helper import * 
# Plots the prediction in jupyter notebook for comparison
i = 90

def compareMasks(prediction, groundTruth):
    
    diff_img = np.zeros((224,224,3), dtype=np.uint8)

    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if ((prediction[x,y] == groundTruth[x,y]) & groundTruth[x,y].all() != False):
                diff_img[x,y] = (255,255,255)
            elif (prediction[x,y] > groundTruth[x,y]):
                diff_img[x,y] = (255,0,0)
            elif (groundTruth[x,y] > prediction[x,y]):
                diff_img[x,y] = (0,255,0)

    return diff_img

def postProcessImage(img):
    img[img > 0.4] = 255
    img[img <= 0.4] = 0
    return img
    
result = postProcessImage(pred[i][:,:,0])
gt = postProcessImage(y[i][:,:,0])
diff = compareMasks(result, gt)

# Input Picture
plt.imshow(x[i, ..., 0], cmap='gray')
plt.axis('off')
plt.title('Eingabebild')
plt.show()

# Ground Truth
plt.imshow(gt, cmap = 'gray')
plt.title('Grundwahrheit')
plt.axis('off')
plt.show()

# Prediction Dim 0
plt.imshow(result, cmap = 'gray')
plt.title('Vorhersage')
plt.axis('off')
plt.show()

# Prediction Dim 1

plt.imshow(diff)
plt.title('Differenz')
plt.axis('off')
plt.show()

IndexError: index 90 is out of bounds for axis 0 with size 19

OutputDir = '/home/dan/Desktop/Datenset_Block3/test/output'
    
imagePath = OutputDir + '/img/' + str(i) + '.png'
maskPath =  OutputDir + '/pred/' + str(i) + '_pred.png'
gtPath =    OutputDir + '/gt/' + str(i) + '_gt.png'
diffPath =  OutputDir + '/diff/' + str(i) + '_diff.png'
    
    
sc.imsave(imagePath, x[i, ... ,0])
sc.imsave(gtPath, gt)
sc.imsave(maskPath, result)
sc.imsave(diffPath, diff)

In [ ]:
'''
for i in range(0, len(x)):    
    
    number = ('0'+ str(i)) if (i < 10) else str(i)
    
    result = postProcessImage(pred[i][:,:,0])
    gt     = postProcessImage(y[i][:,:,0])
    diff   = compareMasks(result, gt)
    
    imagePath = OutputDir + '/img/' + number + '.png'
    maskPath =  OutputDir + '/pred/' + number + '_pred.png'
    gtPath =    OutputDir + '/gt/' + number + '_gt.png'
    diffPath =  OutputDir + '/diff/' + number + '_diff.png'
    
    
    sc.imsave(imagePath, x[i, ... ,0])
    sc.imsave(gtPath, gt)
    sc.imsave(maskPath, result)
    sc.imsave(diffPath, gt)
    
    if (i % 200 == 0):
        print (i)
'''

In [ ]:
import loss_metrics 

print('DSC:       ' + str(f1_np(y, pred)))
print('IoU:       ' + str(iou_np(y, pred)))
print('Precision: ' + str(precision_np(y, pred)))
print('Recall:    ' + str(recall_np(y, pred)))
print('Error:     ' + str(error_np(y, pred)))

In [ ]:
# list all data in history

print(model.history.keys())
# summarize history for accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')                      # Benennung des Grafen
plt.ylabel('accuracy')                           # Benennung der y-Achse
plt.xlabel('epoch')                              # Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')  # Legende
plt.show()

# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')                          # Benennung des Grafen
plt.ylabel('loss')                               # Benennung der y-Achse
plt.xlabel('epoch')                              # Benennung der x-Achse
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(pred.shape)

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(input=model.input, output=layer_outputs)
activations = activation_model.predict(test_image,1)

print(test_image.shape)

def display_activation(activations, col_size, row_size, act_index): 
    
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    
    for row in range(0,row_size): 
        for col in range(0,col_size):
            ax[row][col].imshow(activation[:, :, activation_index], cmap='gray')
            activation_index += 1
            
#display_activation(pred, 6, 5, 0)